In [1]:
import os 
import copy 
import pandas as pd 
import json

In [16]:
#global 
RESULTS_PATH = './results'

DATASETS = {'ActiveMQ':[  
               'activemq-5.0.0', 'activemq-5.1.0',
               'activemq-5.2.0', 'activemq-5.3.0'
            ],
            'Derby':[  
               'derby-10.2.1.6', 'derby-10.3.1.4'
            ],
            'Groovy' : [
                'groovy-1_5_7', 'groovy-1_6_BETA_1',
                ],
            'Hbase' : [
                'hbase-0.94.0', 'hbase-0.95.0'
            ],
            'Hive': [
               'hive-0.9.0', 'hive-0.10.0'
            ],
            'Jruby' : [
                 'jruby-1.1', 'jruby-1.4.0', 'jruby-1.5.0'
            ],
            'Lucene' : [
                 'lucene-2.3.0', 'lucene-2.9.0', 'lucene-3.0.0'
            ],
            'Wicket' : [
       'wicket-1.3.0-incubating-beta-1','wicket-1.3.0-beta2'],
        }
ML_ALGOS = ['DFDP', 'RF','LR','NB', 'DT','FFT']
SBSE_ALGOS = ['MEG', 'MOLR']
OURS = 'nsga3'
TOOL_NAME = 'LessComplexSDP'




In [17]:
#helpers 
def prepare_full_table(metrics_data) : 
    all_results = []
    for metric_data in  metrics_data : 
        all_results.append(results_to_latex(**metric_data))
    final_result = []
    for i in range(len(all_results[0])) : 
        new_line = ''
        for j in range(len(all_results)) : 
            new_line += all_results[j][i]
        new_line  +=  '\\\\'
        final_result.append(new_line)
    return final_result

def results_to_latex(data,significance_level = 0.01,round_digits = 3,
                     projects = [],algorithms = [],main_algo = 'ibea',
                     greater_better = True,performance = 'median', prefix = '',add_project_name = True) : 
    results = []
    for project in projects :
        
        if add_project_name : 
            s = prefix+project+prefix
        else : 
            s = prefix
        #selecting main algo row 
        algos_performances = select_algos_performance_and_statistical_tests(data,algorithms,project,performance)
        best_performance_marked = mark_best_performance(algos_performances,round_digits,greater_better)
        
        added_statistical_interpretation = add_statistical_interpretation(best_performance_marked,significance_level,greater_better)
        for algo in algorithms : 
            if algo == main_algo : 
                if added_statistical_interpretation[algo]['marked'] : 
                    s+= '\\textbf{'+str(added_statistical_interpretation[algo]['performance'])+'}&'
                else :
                    s+= str(added_statistical_interpretation[algo]['performance'])+'&'
            else : 
                if added_statistical_interpretation[algo]['marked'] : 
                    s+= '\\textbf{'+str(added_statistical_interpretation[algo]['performance'])+'} (' + added_statistical_interpretation[algo]['statistical_interpretation'] +')&'
                else :
                    s+= str(added_statistical_interpretation[algo]['performance'])+' (' + added_statistical_interpretation[algo]['statistical_interpretation'] +')&'
        s = s[:-1]
        results.append(s)
    return results
            
def select_algos_performance_and_statistical_tests(data,algos,project,performance) : 
    algos_performances = {}
    for algo in algos : 
        algo_current_row = data[(data['algorithm'] == algo) & (data['file_id'] == project)]
        try:
            algo_current_row = algo_current_row.iloc[0].to_dict()
        except:
            print(algo)
            print(algo_current_row)
            continue
        algos_performances[algo] = {'performance': float(algo_current_row[performance]),'p_value': float(algo_current_row['p_value']),'effect_size': float(algo_current_row['effect_size'])}
    return algos_performances
def add_statistical_interpretation(statistical_tests,significance_level,greater_better) : 
    results = copy.deepcopy(statistical_tests)
    
    for algo in statistical_tests : 
        interpretation = ''
        if statistical_tests[algo]['p_value'] < significance_level :
            if greater_better : 
                if statistical_tests[algo]['effect_size'] > 0 : 
                    interpretation+= '+'
                if statistical_tests[algo]['effect_size'] < 0 : 
                    interpretation+= '-'
            else : 
                if statistical_tests[algo]['effect_size'] > 0 : 
                    interpretation+= '-'
                if statistical_tests[algo]['effect_size'] < 0 : 
                    interpretation+= '+'
        else :
            interpretation += 'o'
        interpretation += cliff_interpretation(abs(statistical_tests[algo]['effect_size']))
        
        results[algo]['statistical_interpretation'] = interpretation
    
    return results 
def mark_best_performance(algos_performances,round_digits,greater_better) : 
    rounded_results = round_algorithms_performance(algos_performances,round_digits) 
    best_result = max([rounded_results[algo]['performance'] for algo in rounded_results]) 
    
    if not greater_better : 
        best_result = min([rounded_results[algo]['performance'] for algo in rounded_results]) 
    
    results = rounded_results
    for algo in results : 
        results[algo]['marked'] = False 
        if rounded_results[algo]['performance'] == best_result : 
            results[algo]['marked'] = True 
    return results 
    
def round_algorithms_performance(algos_performances,round_digits) : 
    result = copy.deepcopy(algos_performances)
    for algo in result.keys() : 
        result[algo]['performance'] = round(result[algo]['performance'], round_digits)
    return result

def cliff_interpretation(value) : 
    if value < 0.147 : 
        return 'N'
    if value >= 0.147 and value < 0.330 : 
        return 'S'
    if value >= 0.330 and value < 0.474 : 
        return 'M'
    if value >=  0.474 : 
        return 'L'

def select_algo_performance(df,org,algo) : 
    selected_row=df[(df['file_id'] == org) & (df['algorithm'] == algo)].to_dict(orient='records')[0]
    return selected_row

## RQ1 MOGP vs ML 

In [18]:
#RQ1 globals 
RQ1_GP_VS_ML_TUNED_WILCOXIN = [
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_G.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_MCC.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_f1.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        }
    ]
RQ1_GP_VS_ML_TUNED_COST_SENSITIVE_WILCOXIN = [
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_TUNED_COST_SENSITIVE_G.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_TUNED_COST_SENSITIVE_MCC.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_TUNED_COST_SENSITIVE_f1.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        }
        
    ]
RQ1_GP_VS_ML_TUNED_SMOTE_WILCOXIN = [
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_SMOTE_G.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_SMOTE_MCC.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','GP_VS_ML_SMOTE_f1.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        }
    ]

    

In [19]:
def RQ1_Table_Wilcoxin(datasets, metrics_data, main_algo ,algos,
                        sig_level = 0.05, apply_bonforoni = False):
    new_sig_level = sig_level
    if apply_bonforoni: 
        new_sig_level = sig_level/len(algos)
    for dataset in datasets : 
        print('dataset:',dataset)
        print('================================================')
        new_metrics_data = copy.deepcopy(metrics_data)
        for metric_data in new_metrics_data: 
            metric_data.update({
            "main_algo": main_algo, 
            'projects' : datasets[dataset], 
            'significance_level': new_sig_level,
            'algorithms': [main_algo] + algos
        } )
            
        all_lines = prepare_full_table(new_metrics_data)
        for ligne in all_lines : 
            print(ligne)

def RQ1_tables(metrics_dfs,orgs = ['Eclipse','Libreoffice','Gerrithub','All'],
               algos = ['NSGA2_LR_AUC_average_churn_recall_median', 'NSGA2_LR_AUC_average_churn_recall_max','GA_AUC_recall_and_churn'],
               ) : 
    
    for org_index,org in enumerate(orgs): 
        org_lines = []
        #print('org:',org,'=====================================================')
        print(f'\\multirow{{{len(algos)}}}{{2cm}}{{{org}}}')
        for index,algo in enumerate(algos): 
            line = "&"
            line +=algo+'&'
            for index,(metric,metric_data) in enumerate(metrics_dfs.items()) :
                model_metric = select_algo_performance(metric_data['data'],org,algo) 
                item = f'{round(model_metric["mean"],metric_data["round_digits"])} ({model_metric["group"]})'
                if model_metric['group'] == 1 : 
                    item = f'\\textbf{{{item}}}'
                if index == len(metrics_dfs)-1 : 
                    item += "\\\\"
                else :
                    item+= "&"
                line +=item
            line = line.replace('NSGA3_LR_recall_TNR_churn_max','\\approach \\_B')
            line = line.replace('NSGA3_LR_recall_TNR_churn_median','\\approach \\_M')
            line = line.replace('GA_AUC','GA\_AUC')
            line = line.replace('GA_recall','GA\_recall')
            line = line.replace('GA_churn','GA\_churn')
            org_lines.append(line)
            print(line)
        print('\\hline')

In [20]:
RQ1_Table_Wilcoxin(DATASETS, RQ1_GP_VS_ML_TUNED_WILCOXIN, main_algo='nsga3',algos =  ML_ALGOS,apply_bonforoni=True)

dataset: ActiveMQ
&\textbf{0.703}&0.563 (+L)&0.431 (oL)&0.639 (oL)&0.665 (oM)&0.464 (oL)&0.485 (oL)&0.345&\textbf{0.376} (-L)&0.289 (oM)&0.256 (oL)&0.282 (oL)&0.266 (oL)&0.31 (oS)&0.394&\textbf{0.402} (oM)&0.28 (oL)&0.317 (oL)&0.338 (oL)&0.291 (oL)&0.325 (oL)\\
&\textbf{0.725}&0.449 (+L)&0.361 (+L)&0.405 (+L)&0.653 (+L)&0.354 (+L)&0.713 (+L)&0.34&0.312 (+L)&0.336 (oS)&0.362 (-L)&\textbf{0.37} (-L)&0.33 (oL)&0.296 (+L)&0.41&0.304 (+L)&0.229 (+L)&0.278 (+L)&\textbf{0.44} (-L)&0.222 (+L)&0.367 (+L)\\
&\textbf{0.758}&0.622 (+L)&0.579 (+L)&0.0 (+L)&0.718 (+L)&0.526 (+L)&0.739 (+L)&0.359&0.351 (+L)&0.373 (-L)&0.0 (+L)&0.297 (+L)&\textbf{0.377} (-L)&0.313 (+L)&0.416&\textbf{0.42} (oS)&0.418 (oS)&0.0 (+L)&0.366 (+L)&0.389 (+L)&0.367 (+L)\\
&0.0&0.613 (oL)&0.581 (oL)&0.644 (oL)&\textbf{0.753} (-L)&0.62 (oL)&0.74 (-L)&-1.0&0.335 (-L)&\textbf{0.337} (-L)&0.309 (-L)&0.266 (-L)&0.271 (-L)&0.248 (-L)&0.114&\textbf{0.377} (-L)&0.374 (-L)&0.352 (-L)&0.263 (-L)&0.317 (-L)&0.248 (-L)\\
dataset: Derby
&0

In [22]:
RQ1_Table_Wilcoxin(DATASETS, RQ1_GP_VS_ML_TUNED_SMOTE_WILCOXIN, main_algo='nsga3',algos =  ML_ALGOS, apply_bonforoni=True)

dataset: ActiveMQ
&\textbf{0.703}&0.546 (+L)&0.64 (oM)&0.355 (oL)&0.695 (oS)&0.337 (+L)&0.345 (oL)&\textbf{0.345}&0.055 (+L)&0.214 (oL)&-0.073 (+L)&0.294 (oL)&-0.101 (+L)&-0.081 (+L)&\textbf{0.394}&0.16 (+L)&0.275 (oL)&0.126 (+L)&0.345 (oL)&0.12 (+L)&0.125 (+L)\\
&0.725&0.533 (+L)&0.533 (+L)&0.447 (+L)&0.687 (+L)&0.616 (+L)&\textbf{0.793} (-L)&0.34&0.346 (oN)&0.354 (oS)&0.262 (+L)&0.371 (-L)&0.266 (+L)&\textbf{0.395} (-L)&0.41&0.378 (+L)&0.382 (+L)&0.287 (+L)&\textbf{0.444} (-L)&0.354 (+L)&0.432 (-L)\\
&\textbf{0.758}&0.666 (+L)&0.669 (+L)&0.649 (+L)&0.072 (+L)&0.674 (+L)&0.662 (+L)&\textbf{0.359}&0.344 (+L)&0.354 (oS)&0.327 (+L)&-0.04 (+L)&0.282 (+L)&0.317 (+L)&0.416&0.422 (-L)&\textbf{0.43} (-L)&0.407 (+L)&0.195 (+L)&0.367 (+L)&0.399 (+L)\\
&0.0&0.675 (oL)&0.662 (oL)&0.666 (oL)&\textbf{0.769} (-L)&0.624 (oL)&0.0 (oM)&-1.0&0.256 (-L)&0.261 (-L)&0.219 (-L)&\textbf{0.276} (-L)&0.175 (oL)&0.0 (oN)&0.114&0.29 (-L)&\textbf{0.298} (-L)&0.255 (-L)&0.26 (-L)&0.221 (-L)&0.114 (oN)\\
dataset: D

### GP VS ML in terms of complexity

In [22]:
RQ1_GP_VS_DT_COMPLEXITY = [
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','RBCA_complexities_total_node_number.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False,
            'greater_better': False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','RBCA_complexities_decision_nodes_count.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False,
            'greater_better': False
        },
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ1_results','RBCA_complexities_max_depth.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False,
            'greater_better' :False
        }
    ]


In [23]:
RQ1_Table_Wilcoxin(DATASETS, RQ1_GP_VS_DT_COMPLEXITY, main_algo='nsga3',algos =  ['DT', 'DT_SMOTE', 'FFT', 'FFT_SMOTE'])

dataset: ActiveMQ
&\textbf{7.0}&14.0 (+L)&39.0 (+L)&\textbf{7.0} (oS)&9.0 (+L)&4.0&6.5 (+L)&19.0 (+L)&\textbf{3.0} (-L)&4.0 (oN)&\textbf{3.0}&\textbf{3.0} (oN)&8.0 (+L)&4.0 (+L)&5.0 (+L)\\
&\textbf{7.0}&11.0 (+L)&54.0 (+L)&\textbf{7.0} (oN)&9.0 (+L)&4.0&5.0 (+L)&26.5 (+L)&\textbf{3.0} (-L)&4.0 (oN)&\textbf{3.0}&\textbf{3.0} (oN)&10.0 (+L)&4.0 (+L)&5.0 (+L)\\
&7.0&13.0 (+L)&45.0 (+L)&9.0 (+L)&\textbf{5.0} (-L)&4.0&6.0 (+L)&22.0 (+L)&4.0 (oN)&\textbf{2.0} (-L)&\textbf{3.0}&\textbf{3.0} (oN)&8.0 (+L)&5.0 (+L)&\textbf{3.0} (oN)\\
&7.0&31.0 (+L)&51.0 (+L)&\textbf{5.0} (-L)&9.0 (+L)&4.0&15.0 (+L)&25.0 (+L)&\textbf{2.0} (-L)&4.0 (oN)&\textbf{3.0}&5.0 (+L)&9.0 (+L)&\textbf{3.0} (oN)&5.0 (+L)\\
dataset: Derby
&7.0&15.0 (+L)&27.0 (+L)&9.0 (+L)&\textbf{5.0} (-L)&4.0&7.0 (+L)&13.0 (+L)&4.0 (oN)&\textbf{2.0} (-L)&\textbf{3.0}&\textbf{3.0} (oN)&5.0 (+L)&5.0 (+L)&\textbf{3.0} (oN)\\
&7.0&29.0 (+L)&33.0 (+L)&9.0 (+L)&\textbf{5.0} (-L)&4.0&14.0 (+L)&16.0 (+L)&4.0 (oN)&\textbf{2.0} (-L)&\textbf{3.0}&5.0

## RQ: GP_COMPLEXITY_VS_GP_NOCOMPLEXITY

### GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_PERFORMANCE

In [11]:
RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_WILCOXIN = [
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_G.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_MCC.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_f1.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        }
    ]

    

In [12]:
def RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_Table_Wilcoxin(datasets, metrics_data, main_algo ,algos,
                        sig_level = 0.05, apply_bonforoni = False):
    new_sig_level = sig_level
    if apply_bonforoni: 
        new_sig_level = sig_level/len(algos)
    for dataset in datasets : 
        print('dataset:',dataset)
        print('================================================')
        new_metrics_data = copy.deepcopy(metrics_data)
        for metric_data in new_metrics_data: 
            metric_data.update({
            "main_algo": main_algo, 
            'projects' : datasets[dataset], 
            'significance_level': new_sig_level,
            'algorithms': [main_algo] + algos
        } )
            
        all_lines = prepare_full_table(new_metrics_data)
        for ligne in all_lines : 
            print(ligne)

In [13]:
RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_Table_Wilcoxin(DATASETS, RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_WILCOXIN, main_algo='nsga3',algos =  ['nsga3_NO_COMPLEXITY'])

dataset: ActiveMQ
&\textbf{0.703}&0.589 (oN)&\textbf{0.345}&0.294 (oM)&\textbf{0.394}&0.325 (+M)\\
&\textbf{0.725}&0.685 (+L)&\textbf{0.34}&0.237 (+L)&\textbf{0.41}&0.308 (+L)\\
&\textbf{0.758}&0.653 (+L)&\textbf{0.359}&0.31 (+L)&\textbf{0.416}&0.393 (+L)\\
&0.0&\textbf{0.559} (oM)&-1.0&\textbf{0.204} (oL)&0.114&\textbf{0.225} (oS)\\
dataset: Derby
&\textbf{0.731}&0.435 (+L)&\textbf{0.433}&0.216 (oS)&\textbf{0.623}&0.508 (+L)\\
&\textbf{0.661}&0.486 (+L)&\textbf{0.302}&0.178 (+L)&\textbf{0.409}&0.295 (+L)\\
dataset: Groovy
&\textbf{0.642}&0.621 (oS)&\textbf{0.273}&0.26 (oS)&\textbf{0.341}&0.31 (oS)\\
&\textbf{0.736}&\textbf{0.736} (oS)&\textbf{0.372}&0.328 (+L)&\textbf{0.422}&0.373 (+L)\\
dataset: Hbase
&\textbf{0.641}&0.418 (+L)&\textbf{0.334}&0.303 (+L)&\textbf{0.486}&0.289 (+L)\\
&\textbf{0.64}&0.474 (+L)&\textbf{0.251}&0.249 (oN)&\textbf{0.49}&0.348 (+L)\\
dataset: Hive
&\textbf{0.762}&0.747 (+L)&\textbf{0.369}&0.333 (+L)&\textbf{0.424}&0.392 (+L)\\
&\textbf{0.716}&0.7 (oS)&\textbf

### GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_PERFORMANCE

In [31]:
RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_WILCOXIN = [
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_COMPLEXITY_total_node_number.csv' )),
            'round_digits' : 0,
            'greater_better' : False,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_COMPLEXITY_decision_nodes_count.csv' )),
            'round_digits' : 0,
            'greater_better' : False,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY','GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_COMPLEXITY_max_depth.csv' )),
            'round_digits' : 0,
            'greater_better' : False,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        }
    ]

    

In [32]:
RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_Table_Wilcoxin(DATASETS, RQ_GP_COMPLEXITY_VS_GP_NOCOMPLEXITY_WILCOXIN, main_algo='nsga3',algos =  ['nsga3_NO_COMPLEXITY'])

dataset: ActiveMQ
&\textbf{9.0}&20.0 (+L)&\textbf{6.0}&12.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&18.0 (+L)&\textbf{6.0}&10.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&30.0 (+L)&\textbf{6.0}&18.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&32.0 (+L)&\textbf{6.0}&18.0 (+L)&\textbf{3.0}&4.0 (+L)\\
dataset: Groovy
&\textbf{9.0}&32.0 (+L)&\textbf{6.0}&18.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&26.0 (+L)&\textbf{6.0}&15.0 (+L)&\textbf{3.0}&4.0 (+L)\\
dataset: Hbase
&\textbf{9.0}&42.0 (+L)&\textbf{6.0}&26.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&21.0 (+L)&\textbf{6.0}&13.0 (+L)&\textbf{3.0}&4.0 (+L)\\
dataset: Hive
&\textbf{9.0}&32.0 (+L)&\textbf{6.0}&20.0 (+L)&\textbf{3.0}&4.0 (+L)\\
&\textbf{9.0}&26.0 (+M)&\textbf{6.0}&16.0 (+M)&\textbf{3.0}&4.0 (+L)\\
dataset: Jruby
&\textbf{9.0}&15.0 (+L)&\textbf{6.0}&9.0 (+L)&\textbf{3.0}&4.0 (+M)\\
&\textbf{9.0}&62.0 (+L)&\textbf{6.0}&37.0 (+L)&\textbf{3.0}&5.0 (+L)\\
&\textbf{9.0}&12.0 (oN)&\textbf{6.0}&7.0 (oN)&\textbf{3.0}&4.0 (+L)\\
d

## RQ: GP_VS OTHER SBSE

In [25]:
RQ_GP_VS_GP_OTHER_SBSE = [
    
        {
            'data':  pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_VS_SBSE','GP_VS_SBSE_G.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_VS_SBSE','GP_VS_SBSE_MCC.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        {
            'data': pd.read_csv(os.path.join(RESULTS_PATH, 'RQ_GP_VS_SBSE','GP_VS_SBSE_f1.csv' )),
            'round_digits' : 3,
            'greater_better' : True,
            'performance' : 'median', 
            'prefix' : '&',
            'add_project_name' : False
        },
        
    ]

    

In [26]:
def RQ_SBSE_VS_GP_Table_Wilcoxin(datasets, metrics_data, main_algo ,algos,
                        sig_level = 0.05, apply_bonforoni = False):
    new_sig_level = sig_level
    if apply_bonforoni: 
        new_sig_level = sig_level/len(algos)
    for dataset in datasets : 
        print('dataset:',dataset)
        print('================================================')
        new_metrics_data = copy.deepcopy(metrics_data)
        for metric_data in new_metrics_data: 
            metric_data.update({
            "main_algo": main_algo, 
            'projects' : datasets[dataset], 
            'significance_level': new_sig_level,
            'algorithms': [main_algo] + algos
        } )
            
        all_lines = prepare_full_table(new_metrics_data)
        for ligne in all_lines : 
            print(ligne)

In [27]:
RQ_SBSE_VS_GP_Table_Wilcoxin(DATASETS, RQ_GP_VS_GP_OTHER_SBSE, main_algo='nsga3',algos =  ['MOLR', 'MEG'],apply_bonforoni=True)

dataset: ActiveMQ
&\textbf{0.703}&0.528 (+L)&0.617 (oM)&\textbf{0.345}&0.114 (+L)&0.287 (oM)&\textbf{0.394}&0.177 (+L)&0.315 (oM)\\
&0.725&\textbf{0.726} (oN)&0.618 (+L)&0.34&0.292 (+L)&\textbf{0.347} (oN)&0.41&0.343 (+L)&\textbf{0.417} (oN)\\
&\textbf{0.758}&0.742 (+L)&0.675 (+L)&\textbf{0.359}&0.316 (+L)&0.333 (+L)&\textbf{0.416}&0.359 (+L)&0.41 (oS)\\
&0.0&0.31 (oN)&\textbf{0.687} (-L)&-1.0&0.064 (oN)&\textbf{0.306} (-L)&0.114&0.122 (oN)&\textbf{0.342} (-L)\\
dataset: Derby
&\textbf{0.731}&0.375 (+L)&0.675 (+L)&\textbf{0.433}&0.14 (+L)&0.384 (+L)&\textbf{0.623}&0.486 (+L)&0.565 (+L)\\
&\textbf{0.661}&0.595 (+L)&0.621 (+L)&0.302&0.211 (+L)&\textbf{0.389} (-L)&0.409&0.322 (+L)&\textbf{0.461} (-L)\\
dataset: Groovy
&\textbf{0.642}&0.631 (oM)&0.629 (oL)&0.273&0.23 (+L)&\textbf{0.328} (oL)&0.341&0.301 (+L)&\textbf{0.39} (oL)\\
&0.736&\textbf{0.773} (-L)&0.461 (+L)&\textbf{0.372}&0.354 (+L)&0.199 (+L)&\textbf{0.422}&0.384 (+L)&0.256 (+L)\\
dataset: Hbase
&0.641&\textbf{0.682} (-L)&0.56 (+

## RQ: Rules Analysis

In [14]:
#globals 
RQ_RESULTS_PATH = './results/RQ_Rules_Analysis'
FEATURES_FREQUENCIES_FILE = 'features_frequencies.json'
ASSOCIATION_RULE_MINING_RESULTS_FILE = 'association_rules_mining.json'

In [15]:
#helpers 
def load_association_rule_mining(path):
    with open(path, "r") as file:
        return json.load(file)

def load_feature_frequency(path): 
    with open(path, "r") as file:
        return json.load(file)

In [22]:
def association_rule_mining_table(dataset, association_rule_mining_results=None, topn = 5): 
    global_line_count = 0 
    for project_name, releases in dataset.items(): 
        if project_name == 'Average':
            continue
        print(f"\multirow{{{len(releases)*topn}}}{{5em}}{{{project_name}}}")
        for release in releases: 
            print(f"&\multirow{{{topn}}}{{5em}}{{{release.split('_')[0].split('-')[-1]}}}&")
            if release=="all":
                continue
            release_name = release.replace("_", "\\_")
            print(f"\multirow{{{topn}}}{{5em}}{{-}}")
            association_rules_for_release = association_rule_mining_results[release]
            
            for i in range(topn):
                if i == 0:
                    line ="&"
                else:
                    line = '&&&'
                if global_line_count % 2 == 0:
                    line+="\\rowcolor[HTML]{DADADA}"
                for item in association_rules_for_release[i]['itemsets']: 
                    line +=f'\\texttt{{{item}}}, '
                line = line[:-2]
                line += f"&{round(association_rules_for_release[i]['support'],3)}\\\\"
                print(line.replace('_','\\_'))
                global_line_count += 1
            print('\\cline{2-5}')
        print('\\hline')

def most_frequent_features_table(dataset, features_frequency=None, percentage = True, topn = 64): 
    global_line_count = 0 
    for project_name, releases in dataset.items(): 
        if project_name == 'Average':
            continue
        print(f"\multirow{{{len(releases)*topn}}}{{5em}}{{{project_name}}}")
        for release in releases: 
            print(f"&\multirow{{{topn}}}{{5em}}{{{release.split('_')[0].split('-')[-1]}}}&")
            if release=="all":
                continue
            release_name = release.replace("_", "\\_")
            print(f"\multirow{{{topn}}}{{5em}}{{-}}")
            file_features_frequency = features_frequency[release]

            if percentage: 
                file_features_frequency = {key: item*100/(sum(file_features_frequency.values())) for key, item in file_features_frequency.items()}
            features = sorted(file_features_frequency.items(), key=lambda x: -x[1])    
            for i in range(topn):
                if i == 0:
                    line ="&"
                else:
                    line = '&&&'
                if global_line_count % 2 == 0:
                    line+="\\rowcolor[HTML]{DADADA}"
                line +=f'\\texttt{{{features[i][0]}}}, '
                line = line[:-2]
                line += f"&{round(features[i][1],2)}\\\\"
                print(line.replace('_','\\_'))
                global_line_count += 1
            print('\\cline{2-5}')
        print('\\hline')


In [17]:
#main
association_rule_mining_results = load_association_rule_mining(os.path.join(RQ_RESULTS_PATH, ASSOCIATION_RULE_MINING_RESULTS_FILE))
features_frequencies = load_feature_frequency(os.path.join(RQ_RESULTS_PATH, FEATURES_FREQUENCIES_FILE))

In [18]:
association_rule_mining_table(DATASETS, association_rule_mining_results)

\multirow{20}{5em}{ActiveMQ}
&\multirow{5}{5em}{5.0.0}&
\multirow{5}{5em}{-}
&\rowcolor[HTML]{DADADA}\texttt{CountStmtExe}, \texttt{OWN\_LINE}&0.033\\
&&&\texttt{CountDeclMethodPrivate}, \texttt{CountStmtDecl}&0.024\\
&&&\rowcolor[HTML]{DADADA}\texttt{CountPath\_Max}, \texttt{OWN\_LINE}&0.02\\
&&&\texttt{CountDeclMethodPrivate}, \texttt{CountLineCodeDecl}&0.017\\
&&&\rowcolor[HTML]{DADADA}\texttt{CountStmtExe}, \texttt{MINOR\_LINE}&0.017\\
\cline{2-5}
&\multirow{5}{5em}{5.1.0}&
\multirow{5}{5em}{-}
&\texttt{CountClassBase}, \texttt{CountSemicolon}&0.025\\
&&&\rowcolor[HTML]{DADADA}\texttt{CountClassBase}, \texttt{CountStmtExe}&0.021\\
&&&\texttt{CountClassBase}, \texttt{CountLineCodeExe}&0.018\\
&&&\rowcolor[HTML]{DADADA}\texttt{CountStmtExe}, \texttt{SumEssential}&0.014\\
&&&\texttt{CountDeclInstanceMethod}, \texttt{CountStmtExe}&0.012\\
\cline{2-5}
&\multirow{5}{5em}{5.2.0}&
\multirow{5}{5em}{-}
&\rowcolor[HTML]{DADADA}\texttt{CountLineCodeDecl}, \texttt{MINOR\_LINE}&0.068\\
&&&\text

IndexError: list index out of range

In [23]:
most_frequent_features_table(DATASETS, features_frequencies,percentage=True,topn=5)

\multirow{20}{5em}{ActiveMQ}
&\multirow{5}{5em}{5.0.0}&
\multirow{5}{5em}{-}
&\rowcolor[HTML]{DADADA}\texttt{CountClassCoupled}&13.77\\
&&&\texttt{CountDeclMethodPrivate}&11.65\\
&&&\rowcolor[HTML]{DADADA}\texttt{MAJOR\_COMMIT}&9.38\\
&&&\texttt{OWN\_LINE}&8.02\\
&&&\rowcolor[HTML]{DADADA}\texttt{DDEV}&7.87\\
\cline{2-5}
&\multirow{5}{5em}{5.1.0}&
\multirow{5}{5em}{-}
&\texttt{ADEV}&15.55\\
&&&\rowcolor[HTML]{DADADA}\texttt{COMM}&11.19\\
&&&\texttt{CountClassBase}&9.16\\
&&&\rowcolor[HTML]{DADADA}\texttt{CountStmtExe}&8.43\\
&&&\texttt{SumCyclomatic}&6.1\\
\cline{2-5}
&\multirow{5}{5em}{5.2.0}&
\multirow{5}{5em}{-}
&\rowcolor[HTML]{DADADA}\texttt{CountLineCodeDecl}&16.08\\
&&&\texttt{CountClassCoupled}&15.02\\
&&&\rowcolor[HTML]{DADADA}\texttt{Added\_lines}&11.08\\
&&&\texttt{MINOR\_LINE}&6.53\\
&&&\rowcolor[HTML]{DADADA}\texttt{SumEssential}&5.16\\
\cline{2-5}
&\multirow{5}{5em}{5.3.0}&
\multirow{5}{5em}{-}
&\texttt{OWN\_LINE}&14.0\\
&&&\rowcolor[HTML]{DADADA}\texttt{SumCyclomaticModi